# Test

This notebook is testing code for writing python implementations of HMMs wiht cuda GPU acceleration.

In [1]:
#activate venv
!source ../.venv/bin/activate

In [2]:
#imports
import numpy as np
import pandas as pd

### Claude 3.5 Sonnet implementation of an HMM
Currently fails miserably!

In [ ]:
import numpy as np

class HiddenMarkovModel:
    def __init__(self, n_states, n_observations):
        """
        Initialize a Hidden Markov Model.
        
        Args:
            n_states: Number of hidden states
            n_observations: Number of possible observations
        """
        # Initialize parameters with random values
        self.n_states = n_states
        self.n_observations = n_observations
        
        # Transition probabilities (A matrix)
        self.A = np.random.random((n_states, n_states))
        self.A = self.A / self.A.sum(axis=1, keepdims=True)
        
        # Emission probabilities (B matrix)
        self.B = np.random.random((n_states, n_observations))
        self.B = self.B / self.B.sum(axis=1, keepdims=True)
        
        # Initial state probabilities (π vector)
        self.pi = np.random.random(n_states)
        self.pi = self.pi / self.pi.sum()

    def forward(self, observations):
        """
        Compute forward probabilities for a sequence of observations.
        
        Args:
            observations: Array-like sequence of observation indices
            
        Returns:
            alpha: Forward probabilities matrix
            likelihood: Probability of the observation sequence
        """
        observations = np.array(observations)
        T = len(observations)
        alpha = np.zeros((T, self.n_states))
        
        # Initialize first step
        alpha[0] = self.pi * self.B[:, observations[0]]
        
        # Forward recursion
        for t in range(1, T):
            for j in range(self.n_states):
                alpha[t, j] = self.B[j, observations[t]] * np.sum(alpha[t-1] * self.A[:, j])
        
        likelihood = np.sum(alpha[-1])
        return alpha, likelihood

    def backward(self, observations):
        """
        Compute backward probabilities for a sequence of observations.
        
        Args:
            observations: Array-like sequence of observation indices
            
        Returns:
            beta: Backward probabilities matrix
        """
        observations = np.array(observations)
        T = len(observations)
        beta = np.zeros((T, self.n_states))
        
        # Initialize last step
        beta[-1] = 1
        
        # Backward recursion
        for t in range(T-2, -1, -1):
            for i in range(self.n_states):
                beta[t, i] = np.sum(self.A[i, :] * self.B[:, observations[t+1]] * beta[t+1])
        
        return beta

    def viterbi(self, observations):
        """
        Find most likely state sequence using Viterbi algorithm.
        
        Args:
            observations: Array-like sequence of observation indices
            
        Returns:
            path: Most likely state sequence
            probability: Probability of the most likely path
        """
        observations = np.array(observations)
        T = len(observations)
        viterbi = np.zeros((T, self.n_states))
        backpointer = np.zeros((T, self.n_states), dtype=int)
        
        # Initialize first step
        viterbi[0] = np.log(self.pi) + np.log(self.B[:, observations[0]])
        
        # Recursion
        for t in range(1, T):
            for j in range(self.n_states):
                temp = viterbi[t-1] + np.log(self.A[:, j])
                viterbi[t, j] = np.max(temp) + np.log(self.B[j, observations[t]])
                backpointer[t, j] = np.argmax(temp)
        
        # Backtrack
        path = [np.argmax(viterbi[-1])]
        for t in range(T-1, 0, -1):
            path.append(backpointer[t, path[-1]])
        path.reverse()
        
        probability = np.exp(np.max(viterbi[-1]))
        return path, probability

    def baum_welch(self, observations, max_iter=100, tol=1e-6):
        """
        Estimate HMM parameters using Baum-Welch algorithm.
        
        Args:
            observations: Array-like sequence of observation indices
            max_iter: Maximum number of iterations
            tol: Convergence tolerance
            
        Returns:
            likelihood: Final log-likelihood
        """
        observations = np.array(observations)
        T = len(observations)
        prev_likelihood = float('-inf')
        
        for _ in range(max_iter):
            # E-step
            alpha, likelihood = self.forward(observations)
            beta = self.backward(observations)
            
            # Compute gammas and xis
            gamma = alpha * beta / likelihood
            xi = np.zeros((T-1, self.n_states, self.n_states))
            
            for t in range(T-1):
                for i in range(self.n_states):
                    for j in range(self.n_states):
                        xi[t, i, j] = (alpha[t, i] * self.A[i, j] * 
                                     self.B[j, observations[t+1]] * beta[t+1, j])
                xi[t] = xi[t] / xi[t].sum()
            
            # M-step
            # Update initial probabilities
            self.pi = gamma[0]
            
            # Update transition probabilities
            self.A = xi.sum(axis=0) / gamma[:-1].sum(axis=0).reshape(-1, 1)
            
            # Update emission probabilities
            for j in range(self.n_states):
                for k in range(self.n_observations):
                    mask = observations == k
                    self.B[j, k] = gamma[mask, j].sum() / gamma[:, j].sum()
            
            # Check convergence
            if abs(likelihood - prev_likelihood) < tol:
                break
            prev_likelihood = likelihood
            
        return likelihood

In [14]:
# Create a simple HMM with 2 states and 3 possible observations
hmm = HiddenMarkovModel(n_states=3, n_observations=5)
    
# Generate some sample observations
observations = [0, 1, 2, 1, 0]
    
# Find most likely state sequence
states, probability = hmm.viterbi(observations)
print(f"Most likely state sequence: {states}")
print(f"Probability: {probability}")
    
# Estimate parameters using Baum-Welch
likelihood = hmm.baum_welch(observations)
print(f"Final log-likelihood: {likelihood}")

Most likely state sequence: [np.int64(1), np.int64(2), np.int64(2), np.int64(0), np.int64(1)]
Probability: 1.2187241269992641e-05
Final log-likelihood: 0.2499999999993308
